In [1]:
import numpy as np
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import matplotlib.pyplot as plt

In [2]:
Learning_Rate = 1e-5
width = height = 200 # image width and height
batchSize = 1

# Create training image

In [3]:
def ReadRandomImage(): 
    FillLevel=np.random.random() # Set random fill level
    Img=np.zeros([900,900,3],np.uint8) # Create black image 
    Img[0:int(FillLevel*900),:]=255 # Fill the image with white up to FillLevel
    
    transformImg=tf.Compose([tf.ToPILImage(), tf.Resize((height,width)), tf.ToTensor(), tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) 
    Img=transformImg(Img) # Transform to pytorch
    return Img,FillLevel

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(10, 10))

for i in range(5):
  for j in range(5):
    image, fill_level = ReadRandomImage()
    image_np = image.permute(1, 2, 0).numpy()  # Convert to numpy array and change dimensions for plotting
    image_np = (image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])).clip(0, 1)  # Denormalize and clip to valid range
    axes[i, j].imshow(image_np)
    axes[i, j].axis('off')

plt.show()


# Load batch of images

In [16]:
def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize, 3, height, width])
    FillLevel = torch.zeros([batchSize])
    for i in range(batchSize):
        images[i], FillLevel[i] = ReadRandomImage()
    return images, FillLevel

# Load and set net and optimizer

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # Set device GPU or CPU where the training will take place
Net = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)  # Load net with default weights
Net.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True)  # Change final layer to predict one value
Net = Net.to(device)

optimizer = torch.optim.Adam(params=Net.parameters(), lr=Learning_Rate)  # Create adam optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1)  # Adaptive learning rate scheduler


# Train

In [ ]:
AverageLoss = np.zeros([50])  # Save average loss for display
all_losses = []  # List to keep track of all losses

for itr in range(100001):  # Training loop
  images, GTFillLevel = LoadBatch()  # Load training batch
  images = torch.autograd.Variable(images, requires_grad = False).to(device)  # Load image
  GTFillLevel = torch.autograd.Variable(GTFillLevel, requires_grad = False).to(device)  # Load Ground truth fill level
  PredLevel = Net(images)  # Make prediction
  Net.zero_grad()
  Loss = torch.abs(PredLevel - GTFillLevel).mean()
  Loss.backward()  # Backpropagate loss
  optimizer.step()  # Apply gradient descent change to weight
  scheduler.step()  # Update learning rate
  AverageLoss[itr % 50] = Loss.data.cpu().numpy()  # Save loss average
  all_losses.append(Loss.data.cpu().numpy())  # Keep track of all losses

  if itr % 5000 == 0:
    print(f"{itr})\tLoss = {Loss.data.cpu().numpy():.5f} AverageLoss = {AverageLoss.mean():.5f}")  # Display loss
    print(f"Learning Rate = {scheduler.get_last_lr()[0]:.5e}")  # Print learning rate

  if itr % 50000 == 0:  # Save model
    print("Saving Model" + str(itr) + ".torch")  # Save model weight
    torch.save(Net.state_dict(), str(itr) + ".torch")


In [ ]:
plt.figure(figsize=(10, 5))

selected_losses = all_losses[100::1000]
plt.plot(selected_losses, label='Loss')
plt.xlabel('Iteration (x1000)')
plt.ylabel('Loss')
plt.title('Loss Across Iterations')
plt.legend()
plt.show()

In [ ]:
# Generate 1000 new images and test the trained model
test_images = torch.zeros([1000, 3, height, width])
test_fill_levels = torch.zeros([1000])

for i in range(1000):
  test_images[i], test_fill_levels[i] = ReadRandomImage()

test_images = test_images.to(device)
test_fill_levels = test_fill_levels.to(device)

# Set the model to evaluation mode
Net.eval()

with torch.no_grad():
  predicted_levels = Net(test_images)

# Calculate the mean absolute error on the test set
mae = torch.abs(predicted_levels - test_fill_levels).mean().item()
print(f"Mean Absolute Error on test set: {mae:.5f}")